<a href="https://colab.research.google.com/github/stancsz/notebook-scripts/blob/main/zillow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml
import yaml

# Define the list of zip codes
zip_codes = [
    "calgary-ab-t3k",
    # Add more zip codes here as needed
]

# Write the list to a YAML file
with open('zip_codes.yaml', 'w') as file:
    yaml.dump(zip_codes, file, default_flow_style=False)

In [2]:
import http.client
import yaml
import json

def read_zip_codes(file_name):
    with open(file_name, 'r') as file:
        return yaml.safe_load(file)

def get_listings(zip_code):
    conn = http.client.HTTPSConnection("zillow56.p.rapidapi.com")
    headers = {
        'X-RapidAPI-Key': "",
        'X-RapidAPI-Host': "zillow56.p.rapidapi.com"
    }
    conn.request("GET", f"/search?location={zip_code}&status=forSale&price_min=200000&price_max=750000", headers=headers)
    res = conn.getresponse()
    data = res.read()
    # print(data.decode("utf-8"))
    return json.loads(data.decode("utf-8"))

def format_to_markdown(listings):
    # Calculate price per square foot and add it to each listing
    for listing in listings['results']:
        if listing['livingArea'] > 0:  # To avoid division by zero
            listing['pricePerSqFt'] = listing['price'] / listing['livingArea']
        else:
            listing['pricePerSqFt'] = float('inf')  # Set to infinity if living area is zero

    # Sort listings by price per square foot
    sorted_listings = sorted(listings['results'], key=lambda x: x['pricePerSqFt'])

    # Format sorted listings into Markdown
    markdown = ""
    for listing in sorted_listings:
        # Construct Zillow URL
        address_url = f"https://www.zillow.com/homes/{listing['streetAddress'].replace(' ', '%20')}%20{listing['city'].replace(' ', '%20')}%2C%20{listing['state']}%20{listing['zipcode']}"

        markdown += f"- **Address**: [{listing['streetAddress']} {listing['city']}, {listing['state']} {listing['zipcode']}]({address_url})\n"
        markdown += f"  - **Price**: {listing['price']} {listing['currency']}\n"
        markdown += f"  - **Price per SqFt**: {listing['pricePerSqFt']:.2f} {listing['currency']}/sqft\n"
        markdown += f"  - **Bedrooms**: {listing['bedrooms']}, **Bathrooms**: {listing['bathrooms']}\n"
        markdown += f"  - **Type**: {listing['homeType']}\n"
        markdown += f"  - **Area**: {listing['livingArea']} sqft\n"
        markdown += f"  - **Image**: ![Image]({listing['imgSrc']})\n\n"
    return markdown

# Main program
zip_codes = read_zip_codes("zip_codes.yaml")  # Assuming your YAML file is named zip_codes.yaml
for zip_code in zip_codes:
    listings = get_listings(zip_code)
    markdown = format_to_markdown(listings)
    print(markdown)


- **Address**: [17 SE Country Village Bay NE #112 Calgary, AB T3K5Z3](https://www.zillow.com/homes/17%20SE%20Country%20Village%20Bay%20NE%20#112%20Calgary%2C%20AB%20T3K5Z3)
  - **Price**: 249900.0 CAD
  - **Price per SqFt**: 294.00 CAD/sqft
  - **Bedrooms**: 2.0, **Bathrooms**: 1.0
  - **Type**: APARTMENT
  - **Area**: 850.0 sqft
  - **Image**: ![Image](https://photos.zillowstatic.com/fp/c54142c1dbf02bf08fb07266d1a61032-p_e.jpg)

- **Address**: [125 S Panatella Way NW #1713 Calgary, AB T3K0R9](https://www.zillow.com/homes/125%20S%20Panatella%20Way%20NW%20#1713%20Calgary%2C%20AB%20T3K0R9)
  - **Price**: 450000.0 CAD
  - **Price per SqFt**: 299.60 CAD/sqft
  - **Bedrooms**: 3.0, **Bathrooms**: 3.0
  - **Type**: TOWNHOUSE
  - **Area**: 1502.0 sqft
  - **Image**: ![Image](https://photos.zillowstatic.com/fp/1b710bdee2ec0249852ab5376cf543f3-p_e.jpg)

- **Address**: [388 N Sandarac Dr NW #115 Calgary, AB T3K4E3](https://www.zillow.com/homes/388%20N%20Sandarac%20Dr%20NW%20#115%20Calgary%2C%20A